In [1]:
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama import OllamaLLM

# 載入環境變數
load_dotenv()

# 建立Ollama模型
model = OllamaLLM(model="llama3.2:latest")
print("模型已建立")

模型已建立


In [2]:
# 第一步:生成郵件回覆
reply_prompt = ChatPromptTemplate.from_messages([
    ("system","你是專業的客服代表。請針對客戶的問題提供簡潔、專業的回覆內容。"),
    ("human","客戶郵件：\n\n{email_content}")
])

# 第二步:格式化郵件
format_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "請將以下郵件回覆格式化，加上適當的稱呼、問候語和專業的結尾。"),
        ("human", "郵件內容：\n\n{reply}")
    ]
)

# 第三步: 品質檢查和改進建議
improve_prompt = ChatPromptTemplate.from_messages([
    ("system","是郵件品質專家。請評估這封郵件的專業度、友善度和完整度（1-10分），並提供具體的改進建議。"),
    ("human", "評估並改進這封郵件：\n\n{formatted_reply}")
])

print("三階段 prompt已定義完成")

三階段 prompt已定義完成


In [3]:
sequential_chain = (
    reply_prompt
    | model
    | StrOutputParser()
    | (lambda x:{"reply":x})
    | format_prompt
    | model
    | StrOutputParser()
    | (lambda x: {"formatted_reply":x})
    | improve_prompt
    | model
    | StrOutputParser()
)
print("✅ 串聯模型鏈已建立完成！")
print("\n流程：生成回覆 → 格式化郵件 → 品質評估與改進建議")

✅ 串聯模型鏈已建立完成！

流程：生成回覆 → 格式化郵件 → 品質評估與改進建議


In [5]:
customer_email = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞。
請問可以退貨或換貨嗎？

王小明
"""

result = sequential_chain.invoke({"email_content":customer_email})
print(result)

以下是評估和改進這封郵件的結果：

**專業度：8/10**

* 專業化的 formatization 和字體排版是優點，但缺少了一定的友善感和人性化的元素。
* 回覆郵件的內容清晰易懂，但有一些術語可能需要更常見的人群了解（例如“商品卡”）。
* 專業度可以更好地提高，考慮使用簡單且明確的語言。

**友善度：7/10**

* 回覆郵件的tone 主要是正確和有序，但缺少了一定的溫暖和人性化的元素。
* 使用一些關鍵字，如“歉意”和“致歉”，可以增加友善感，但仍需要更多的體現人性的細節。
* 對客服代表的 introduction 可能可以更加親切地開展。

**完整度：9/10**

* 回覆郵件的內容全方位 COVER了退貨政策、 Replacement produit 等相關資訊。
* 有關任何疑問或需求的建議非常合理，對客服的幫助很有幫助。

改進建議：

1.  在 introduction 中加入一些簡單的自我介紹，並表達對王小明的尊重和好感度。例如："尊敬的王小明，請允许我們這次機會與您聯繫，希望能夠提供正確的幫助。”
2.  將術語中可能不常見的詞彙（如“商品卡”) 進行簡單的解釋和具體範例的表述，讓王小明更容易理解。例如："如果您需要換貨，請提供新購買的收據，並我們會盡快將 replacement sản phẩm寄送至您."
3.  在感謝部分，可以增加一些友善的元素，例如表達對王小明的理解和歉意的幫助。例如："再次深表歉意讓您遇到的任何不便，我们希望能夠提供更好的服務."


In [6]:
#分步執行以展示每個階段的輸出
print("="*60)
print("階段1️⃣:生成初步回覆")
print("="*60)

step1_chain = reply_prompt | model | StrOutputParser()
step1_result = step1_chain.invoke({"email_content":customer_email})
print(step1_result)

print("="*60)
print("階段2️⃣:格式化郵件")
print("="*60)
step2_chain = format_prompt | model | StrOutputParser()
step2_result = step2_chain.invoke({"reply":step1_result})
print(step2_result)

print("="*60)
print("階段3️⃣:品質評估與改進建議")
print("="*60)
step3_chain = improve_prompt | model | StrOutputParser()
step3_result = step3_chain.invoke({"formatted_reply":step2_result})

print(step3_result)

階段1️⃣:生成初步回覆
尊敬的王小明,

感謝您的寄 letter。很抱歉您收到的包裝有損壞，致使您的購買品質受到影響。

根據我們的退貨政策，您可以將產品回送給我們，並享受退貨補償或換貨的選擇。具體流程如下：

1. 請在信件中附上 product 的保留證明和 photograph 以及包裝有損壞的照片，讓我們確認您的購買信息。
2.  您可以選擇退貨或換貨。
3.  退貨後，您將得到原價金額的補償。
4.  换貨，產品將在您提供的地址送到。

如果您需要更詳細的信息或有任何問題，请請不 hesitate聯繫我們。 chúng tôi期待您的回覆。

感謝您的信件，並感謝您的支持。

best regards,
[你的客服代表]
階段2️⃣:格式化郵件
尊敬的王小明：

問候!

感謝您寄來的信件，我们非常抱歉因為包裝有損壞，導致您的購買品質受到影響。我们希望能夠提供您一個好的購買體驗。

根據我們的退貨政策，您可以選擇將產品回送給我們，並享受退貨補償或換貨的選擇。具體流程如下：

1.  請在信件中附上 product 的保留證明和 photograph 以及包裝有損壞的照片，讓我們確認您的購買信息。
2.  您可以選擇退貨或換貨。
3.  退貨後，您將得到原價金額的補償。
4.  换貨，產品將在您提供的地址送到。

如果您需要更詳細的信息或有任何問題，请請不 hesitate聯繫我們。 chúng tôi期待您的回覆，並感謝您的支持。

看法和合作，

[你的客服代表]
階段3️⃣:品質評估與改進建議
根據評估結果，我給出了以下分數：

專業度：8/10

友善度：9/10

完整度：9/10

建議改進：

1.  專業度：雖然信件內容有關注購買品質問題，但是可以更明確地針對特定問題表達，並提供具體的解決方案，例如："因為包裝有損壞，您可以選擇退貨或換貨。退貨後您將得到原價金額的補償." 的句子中，可以更明確地表達退貨和補償的細節。
2. 友善度：信件開始有"問候!"，表達了客服對王小明的尊重，但是可以加上更多的情感性表達，例如"我們非常抱歉因為包裝有損壞，導致您的購買品質受到影響。" 的句子中，可以更明確地表達客服的道歉意願，並與王小明建立更好的關係。
3. 完整度：信件內容全面，但是可以增加一些詳細信息，例如退貨流程中需要填寫什麼表格，以及包裹到哪個